<a href="https://colab.research.google.com/github/ShiqiZ1208/impact-of-loss-function-on-BERT-sentence-embedding/blob/main/example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/SentenceEmbedding

/content/drive/MyDrive/SentenceEmbedding


In [2]:
!ls

bert_sts_results.npy  prepare_STS.py  requirement.txt
Loss_Functions.py     __pycache__     TrainSE.py


In [3]:
!pip install -U -r requirement.txt

In [4]:
from prepare_STS import get_sts_dataset
data_name = "STS-B"
datasets1 = get_sts_dataset(data_name) # load STS-B datasets
print(f"the detial infromation of {data_name} datasets:\n{datasets1}\n")
print(f"first training data pair in {data_name} datasets: \nsentence 1: {datasets1[0][0]['sentence1']} \nsetence 2: {datasets1[0][0]['sentence2']}\nlabel: {datasets1[0][0]['labels']}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


the detial infromation of STS-B datasets:
(Dataset({
    features: ['split', 'genre', 'dataset', 'year', 'sid', 'labels', 'sentence1', 'sentence2'],
    num_rows: 965
}), Dataset({
    features: ['split', 'genre', 'dataset', 'year', 'sid', 'labels', 'sentence1', 'sentence2'],
    num_rows: 414
}))

first training data pair in STS-B datasets: 
sentence 1: You can do it, too. 
setence 2: Yes, you can do it.
label: 5.0


In [8]:
from transformers import AutoModel, AutoTokenizer
model_id = 'bert-base-uncased'
#get pretrain model on huggingface
model = AutoModel.from_pretrained(model_id)
#get the tokenizer for pretrain BERT
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [10]:
sentences = datasets1[0][0]['sentence1']

In [13]:
encodings = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True).to("cpu")
embeddings = model(**encodings).last_hidden_state[:, 0, :]

In [17]:
sentences2 = datasets1[0][0]['sentence2']

In [18]:
encodings2 = tokenizer(sentences2, return_tensors='pt', padding=True, truncation=True).to("cpu")
embeddings2 = model(**encodings2).last_hidden_state[:, 0, :]

In [21]:
print(f"The embeddings for sentence ({datasets1[0][0]['sentence1']}) is: \n{embeddings}")
print(f"The embeddings for sentence ({datasets1[0][0]['sentence2']}) is: \n{embeddings2}")

The embeddings for sentence (You can do it, too.) is: 
tensor([[ 2.5566e-01,  8.5179e-02, -3.1172e-01, -3.1991e-02, -4.4751e-01,
         -1.6575e-01,  4.6797e-01,  3.6699e-01, -2.1939e-02, -2.9789e-01,
          1.3336e-01, -2.6503e-01,  4.3632e-02,  2.8125e-01,  2.3755e-01,
         -3.2472e-01, -2.1934e-01,  4.7536e-01,  3.0173e-01, -7.0426e-02,
         -6.7046e-02, -2.4225e-01,  6.1829e-02, -3.2574e-02, -2.3211e-01,
         -2.4381e-01, -2.1980e-02, -3.0008e-02,  1.8772e-01, -1.6127e-01,
          1.3977e-03,  2.9594e-01, -3.3065e-01, -1.8896e-01,  5.0593e-01,
          7.3877e-02, -5.1107e-02,  8.9503e-02,  1.6768e-01,  4.6468e-02,
         -1.8382e-01,  1.4873e-01, -4.9079e-02,  3.2143e-01,  5.6315e-01,
         -4.8745e-01, -2.5552e+00, -7.8371e-02, -1.2088e-01, -4.1333e-01,
          3.0987e-01,  8.0328e-02,  2.7444e-01,  3.5146e-02,  1.3345e-01,
          1.1566e-01, -3.1539e-01,  8.0275e-01,  7.1067e-02,  4.9331e-01,
         -2.6429e-01,  2.8937e-01,  1.6064e-01, -6.1032e-

In [23]:
import torch.nn.functional as F
cosine_similarity = F.cosine_similarity(embeddings, embeddings2, dim=1)
print(cosine_similarity)

tensor([0.9577], grad_fn=<SumBackward1>)


In [ ]:
!python TrainSE.py

Running: without_ft on STS-B
2025-06-11 21:02:42.584343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749675762.603435   16039 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749675762.609346   16039 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-11 21:02:42.628867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loop 0 spearman - 0.2736426234060464
Loop 1 spearman - 0.20164196854728297
Loop 2 spearman - 0.2

In [ ]:
import numpy as np

data = np.load('bert_sts_results.npy', allow_pickle=True)  # Replace with your actual file name
print(data)

[{'loss': 'without_ft', 'dataset': 'STS-B', 'spearman': np.float64(0.2348376893694606)}
 {'loss': 'without_ft', 'dataset': 'STS12', 'spearman': np.float64(0.2166663085738433)}
 {'loss': 'without_ft', 'dataset': 'STS13', 'spearman': np.float64(0.27818345673950035)}
 {'loss': 'without_ft', 'dataset': 'STS14', 'spearman': np.float64(0.2342810556153155)}
 {'loss': 'without_ft', 'dataset': 'STS15', 'spearman': np.float64(0.3725269177834342)}
 {'loss': 'without_ft', 'dataset': 'STS16', 'spearman': np.float64(0.4896506258454359)}
 {'loss': 'without_ft', 'dataset': 'SICK-R', 'spearman': np.float64(0.42855942977887507)}
 {'loss': 'cosine_similarity_mse_loss', 'dataset': 'STS-B', 'spearman': np.float64(0.3794748646951967)}
 {'loss': 'cosine_similarity_mse_loss', 'dataset': 'STS12', 'spearman': np.float64(0.27722725704425066)}
 {'loss': 'cosine_similarity_mse_loss', 'dataset': 'STS13', 'spearman': np.float64(0.5274457607496896)}
 {'loss': 'cosine_similarity_mse_loss', 'dataset': 'STS14', 'spearma